In [1]:
import pandas as pd
import gc
import numpy as np

In [2]:
# reemplazar con la dirección del archivo descargado
products = pd.read_csv('C:\\Users\\Usuario\\Downloads\\archive\\products.csv') 
train_orders = pd.read_csv('C:\\Users\\Usuario\\Downloads\\archive\\order_products__train.csv', nrows = 500000)
# las ordenes prior tienen mas de 30 millones de filas, para reducir intentamos con 1 millon
prior_orders = pd.read_csv('C:\\Users\\Usuario\\Downloads\\archive\\order_products__prior.csv', nrows= 3000000) 
orders = pd.read_csv('C:\\Users\\Usuario\\Downloads\\archive\\orders.csv')
#aisles = pd.read_csv('C:\\Users\\Usuario\\Downloads\\archive\\aisles.csv')
#departments = pd.read_csv('C:\\Users\\Usuario\\Downloads\\archive\\departments.csv')

In [3]:
print(orders.columns)

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')


In [4]:
print(orders.columns)
print(prior_orders.columns)

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')


In [5]:
orders_past = orders[orders["eval_set"] == "prior"]
orders_past = orders_past.drop(["eval_set"], axis=1)
# juntamos toda la información de cada orden en un data frame
prior = prior_orders.merge(orders_past, on="order_id", how="left")

In [6]:
# usamos estas funciones para ahorrar espacio
# cuando no necesitamos más un data frame, lo borramos
del prior_orders, orders_past
gc.collect()

23

In [7]:
orders_present = orders[orders["eval_set"]== "train"]
orders_present = orders_present.drop(["eval_set"], axis=1)
present = train_orders.merge(orders_present, on="order_id", how="left")

In [8]:
del train_orders, orders, orders_present
gc.collect()

0

In [9]:
# vemos los usuarios que tenemos en comun entre las ordenes pasada y las actuales
common_users = set(present['user_id']).intersection(set(prior['user_id']))

In [10]:
# nos quedamos solo con esos usuarios (solo usar si no se quiere intentar predecir en usuarios nuevos)
present = present[present['user_id'].isin(common_users)]
prior = prior[prior['user_id'].isin(common_users)]

In [11]:
print(len((present['user_id']).unique()))
print(len((prior['user_id']).unique()))
print(len(common_users))

28976
28976
28976


In [12]:
# para saber las dimensiones de nuestro data frame 
# habrán cambiado si es que nos quedamos con los usuarios en común solamente
print(present.shape)
print(prior.shape)

(310377, 9)
(689200, 9)


In [13]:
# Count how many times each user-product pair appears
user_product_counts = prior.groupby(['user_id', 'product_id']).size()

# Identify first-time purchases (should be where count == 1 and reordered == 0)
first_time_purchases = prior[(prior['reordered'] == 0)]

# See if there are any reordered = 0 where the user has bought the product before
# (i.e., check for any inconsistencies)
reordered_zero_duplicates = first_time_purchases.duplicated(subset=['user_id', 'product_id'], keep=False)

print(reordered_zero_duplicates.any())  # Should be False if reordered=0 means first-time


False


In [14]:
# estas columnas podrían ser muy influyentes
# la mayoría de nuestros negativos serán combinaciones de usuario-producto no existentes previamente

present = present.drop('reordered', axis = 1)
prior = prior.drop('reordered', axis = 1)

In [15]:
print(present.columns)
print(prior.columns)

Index(['order_id', 'product_id', 'add_to_cart_order', 'user_id',
       'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order'],
      dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'user_id',
       'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order'],
      dtype='object')


In [16]:
unique_products_per_user = prior.groupby('user_id')['product_id'].nunique()

# Compute the average number of unique products per user
average_unique_products = unique_products_per_user.mean()

print("Average unique products per user:", average_unique_products)

Average unique products per user: 19.09224875759249


In [17]:
unique_orders_per_user = prior.groupby('user_id')['order_id'].nunique()

# Compute the average number of unique products per user
average_unique_orders = unique_orders_per_user.mean()

print("Average orders products per user:", average_unique_orders)

Average orders products per user: 2.3674765323025952


In [18]:

def sample_negatives_per_order(orders_df, all_products, n_neg):
    user_to_products = orders_df.groupby('user_id')['product_id'].apply(set).to_dict()

    # Get one context row per order
    order_context = orders_df.drop_duplicates('order_id').set_index('order_id')[
        ['user_id', 'order_hour_of_day', 'order_dow', 'add_to_cart_order', 'days_since_prior_order']
    ].to_dict('index')

    negatives = []

    for order_id, context in order_context.items():
        user = context['user_id']
        bought_prods = user_to_products[user]
        candidates = list(all_products - bought_prods)
        if not candidates:
            continue
        sampled = np.random.choice(candidates, size=min(n_neg, len(candidates)), replace=False)
        for prod in sampled:
            negatives.append({
                'order_id': order_id,
                'product_id': prod,
                'add_to_cart_order': context['add_to_cart_order'],
                'user_id': user,
                'order_dow': context['order_dow'],
                'order_hour_of_day': context['order_hour_of_day'],
                'days_since_prior_order': context['days_since_prior_order'],
                'label': 0
            })

    return pd.DataFrame(negatives)

In [19]:
# Mark positive examples
positive = prior
positive['label'] = 1
pos_df = present
pos_df['label'] = 1

# All products in catalog
all_products = set(products['product_id'].unique())

# Generate negatives
negative = sample_negatives_per_order(prior, all_products, n_neg=5)
neg_df = sample_negatives_per_order(present, all_products, n_neg=5)

# Combine positive and negative samples
combined = pd.concat([positive, negative], ignore_index=True)
comb_df = pd.concat([pos_df, neg_df], ignore_index=True)


In [20]:
del pos_df, products, neg_df, present, positive, negative
gc.collect()

0

In [21]:

# Product features
product_feats = prior.groupby('product_id').agg(
    prod_total_orders=('order_id', 'count'),
    prod_common_cart_position=('add_to_cart_order', lambda x: x.mode().iloc[0]),
    prod_common_hour=('order_hour_of_day', lambda x: x.mode().iloc[0]),
    prod_avg_hour=('order_hour_of_day', 'mean'),
    prod_std_hour=('order_hour_of_day', 'std'),
    prod_common_day=('order_dow', lambda x: x.mode().iloc[0])
).reset_index()

In [22]:
# features de interacciones entre usuario y producto
user_prod_feats = prior.groupby(['user_id', 'product_id']).agg(
    most_common_day=('order_dow', lambda x: x.mode().iloc[0]),
    common_hour = ('order_hour_of_day', lambda x: x.mode().iloc[0]),
    avg_hour = ('order_hour_of_day', 'mean'),
    std_hour = ('order_hour_of_day', 'std'),
    order_count=('order_id', 'count'),
    common_cart_pos=('add_to_cart_order', lambda x: x.mode().iloc[0]),
    avg_days_between_orders=('days_since_prior_order', 'mean'),
    std_days_between_orders=('days_since_prior_order', 'std')
).reset_index()

In [23]:
comb_df = comb_df.merge(user_prod_feats, on=['user_id', 'product_id'], how='left')
comb_df = comb_df.merge(product_feats, on='product_id', how='left')

In [24]:
combined = combined.merge(user_prod_feats, on=['user_id', 'product_id'], how='left')
combined = combined.merge(product_feats, on='product_id', how='left')

In [25]:
del user_prod_feats, product_feats
gc.collect()

0

In [26]:
comb_df.fillna(0, inplace=True)

In [27]:
combined.fillna(0, inplace=True)  # reemplazar con 0 los valores vacíos donde nunca hubo tal combinación de usuario y producto

In [28]:
# creamos nuevas features que dan una nota en base a que tan similares son 
# las características de ordenes pasadas a las características de las actuales

# features de usuario-producto
comb_df['match_common_hour'] = abs(comb_df['order_hour_of_day'] - comb_df['common_hour']).astype(int)
comb_df['match_days_between'] = abs(comb_df['days_since_prior_order'] - comb_df['avg_days_between_orders']).astype(int)
comb_df['within_days_std'] = (
    (comb_df['days_since_prior_order'] >= comb_df['avg_days_between_orders'] - comb_df['std_days_between_orders']) &
    (comb_df['days_since_prior_order'] <= comb_df['avg_days_between_orders'] + comb_df['std_days_between_orders'])
).astype(int)
comb_df['match_most_common_day'] = abs(comb_df['order_dow'] - comb_df['most_common_day']).astype(int)
comb_df['match_common_cart_pos'] = abs(comb_df['add_to_cart_order'] - comb_df['common_cart_pos']).astype(int)
comb_df['match_avg_hour'] = abs(comb_df['order_hour_of_day'] - comb_df['avg_hour']).astype(int)
comb_df['within_hour_std'] = (
    (comb_df['order_hour_of_day'] >= comb_df['avg_hour'] - comb_df['std_hour']) &
    (comb_df['order_hour_of_day'] <= comb_df['avg_hour'] + comb_df['std_hour'])
).astype(int)
comb_df['user_strength'] = comb_df['order_count'] / comb_df['order_count'].max()


# features específicas al producto
comb_df['match_prod_common_cart_pos'] = abs(comb_df['add_to_cart_order'] - comb_df['prod_common_cart_position']).astype(int)
comb_df['match_prod_common_hour'] = abs(comb_df['order_hour_of_day'] - comb_df['prod_common_hour']).astype(int)
comb_df['match_prod_avg_hour'] = abs(comb_df['order_hour_of_day'] - comb_df['prod_avg_hour']).astype(int)
comb_df['match_prod_common_day'] = abs(comb_df['order_dow'] - comb_df['prod_common_day']).astype(int)
comb_df['within_prod_hour_std'] = (
    (comb_df['order_hour_of_day'] >= comb_df['prod_avg_hour'] - comb_df['prod_std_hour']) &
    (comb_df['order_hour_of_day'] <= comb_df['prod_avg_hour'] + comb_df['prod_std_hour'])
).astype(int)
comb_df['product_strength'] = comb_df['prod_total_orders'] / comb_df['prod_total_orders'].max()

In [29]:
# creamos nuevas features que dan una nota en base a que tan similares son 
# las características de ordenes pasadas a las características de las actuales

# features de usuario-producto
combined['match_common_hour'] = abs(combined['order_hour_of_day'] - combined['common_hour']).astype(int)
combined['match_days_between'] = abs(combined['days_since_prior_order'] - combined['avg_days_between_orders']).astype(int)
combined['within_days_std'] = (
    (combined['days_since_prior_order'] >= combined['avg_days_between_orders'] - combined['std_days_between_orders']) &
    (combined['days_since_prior_order'] <= combined['avg_days_between_orders'] + combined['std_days_between_orders'])
).astype(int)
combined['match_most_common_day'] = abs(combined['order_dow'] - combined['most_common_day']).astype(int)
combined['match_common_cart_pos'] = abs(combined['add_to_cart_order'] - combined['common_cart_pos']).astype(int)
combined['match_avg_hour'] = abs(combined['order_hour_of_day'] - combined['avg_hour']).astype(int)
combined['within_hour_std'] = (
    (combined['order_hour_of_day'] >= combined['avg_hour'] - combined['std_hour']) &
    (combined['order_hour_of_day'] <= combined['avg_hour'] + combined['std_hour'])
).astype(int)
combined['user_strength'] = combined['order_count'] / combined['order_count'].max()


# features específicas al producto
combined['match_prod_common_cart_pos'] = abs(combined['add_to_cart_order'] - combined['prod_common_cart_position']).astype(int)
combined['match_prod_common_hour'] = abs(combined['order_hour_of_day'] - combined['prod_common_hour']).astype(int)
combined['match_prod_avg_hour'] = abs(combined['order_hour_of_day'] - combined['prod_avg_hour']).astype(int)
combined['match_prod_common_day'] = abs(combined['order_dow'] - combined['prod_common_day']).astype(int)
combined['within_prod_hour_std'] = (
    (combined['order_hour_of_day'] >= combined['prod_avg_hour'] - combined['prod_std_hour']) &
    (combined['order_hour_of_day'] <= combined['prod_avg_hour'] + combined['prod_std_hour'])
).astype(int)
combined['product_strength'] = combined['prod_total_orders'] / combined['prod_total_orders'].max()

In [31]:
# usar para ver que dataframes están en la memoria en el momento, su peso y sus dimensiones
# en este punto solo se necesita df
for name, obj in globals().items():
    if isinstance(obj, pd.DataFrame):
        print(f"\n===== DataFrame: {name} =====")
        print(f"Shape: {obj.shape}")
        print(f"Memory usage: {obj.memory_usage().sum() / 1024**2:.2f} MB")


===== DataFrame: prior =====
Shape: (689200, 9)
Memory usage: 52.58 MB

===== DataFrame: first_time_purchases =====
Shape: (281995, 9)
Memory usage: 21.51 MB

===== DataFrame: combined =====
Shape: (1032200, 37)
Memory usage: 291.38 MB

===== DataFrame: comb_df =====
Shape: (455257, 37)
Memory usage: 128.51 MB


In [32]:
print(comb_df.columns) # vemos las columnas

Index(['order_id', 'product_id', 'add_to_cart_order', 'user_id',
       'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'label', 'most_common_day', 'common_hour',
       'avg_hour', 'std_hour', 'order_count', 'common_cart_pos',
       'avg_days_between_orders', 'std_days_between_orders',
       'prod_total_orders', 'prod_common_cart_position', 'prod_common_hour',
       'prod_avg_hour', 'prod_std_hour', 'prod_common_day',
       'match_common_hour', 'match_days_between', 'within_days_std',
       'match_most_common_day', 'match_common_cart_pos', 'match_avg_hour',
       'within_hour_std', 'user_strength', 'match_prod_common_cart_pos',
       'match_prod_common_hour', 'match_prod_avg_hour',
       'match_prod_common_day', 'within_prod_hour_std', 'product_strength'],
      dtype='object')


In [33]:
# quitamos todas las columnas menos las que dan un puntaje de similitud
train_df = comb_df.drop(['order_id', 'add_to_cart_order', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'most_common_day', 'common_hour',
       'avg_hour', 'std_hour', 'order_count', 'common_cart_pos',
       'avg_days_between_orders', 'std_days_between_orders', 'prod_total_orders',
       'prod_common_cart_position', 'prod_common_hour', 'prod_avg_hour',
       'prod_std_hour', 'prod_common_day', 'order_number'], axis = 1)
del comb_df
gc.collect()

752

In [34]:
training_df = combined.drop(['order_id', 'add_to_cart_order', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'most_common_day', 'common_hour',
       'avg_hour', 'std_hour', 'order_count', 'common_cart_pos',
       'avg_days_between_orders', 'std_days_between_orders', 'prod_total_orders',
       'prod_common_cart_position', 'prod_common_hour', 'prod_avg_hour',
       'prod_std_hour', 'prod_common_day', 'order_number'], axis = 1)
del combined
gc.collect()

0

In [ ]:
print(train_df.columns)

In [35]:
del first_time_purchases
gc.collect()

0

In [ ]:
# si uno quiere que solo tomando en cuenta las características del producto
#training_df = training_df.drop(['match_common_hour',
#       'match_days_between', 'within_days_std', 'match_most_common_day',
#       'match_common_cart_pos', 'match_avg_hour', 'within_hour_std'],axis = 1)

In [36]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

meta_train = training_df[['user_id', 'product_id']].copy()
features = [col for col in training_df.columns if col not in ['product_id', 'label']]
X_train = training_df[features]
y_train = training_df['label']

meta_val = train_df[['user_id', 'product_id']].copy()
feats = [col for col in train_df.columns if col not in ['product_id', 'label']]
X_val = train_df[feats]
y_val = train_df['label']


# Group arrays: how many rows per user
train_groups = meta_train.groupby('user_id').size().tolist()
val_groups = meta_val.groupby('user_id').size().tolist()

# Create LightGBM Datasets
train_data = lgb.Dataset(X_train, label=y_train, group=train_groups)
val_data = lgb.Dataset(X_val, label=y_val, group=val_groups)

# LightGBM ranking parameters
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',  # Or 'map' if you prefer
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'num_leaves': 63,
    'learning_rate': 0.05,
    'ndcg_eval_at': [1, 3, 5, 10]  # Common NDCG cutoff values
}

# Train the model
model = lgb.train(
    params,
    train_data,
    valid_sets=[val_data],
    callbacks=[
        lgb.early_stopping(stopping_rounds=40),
        lgb.log_evaluation(period=10)
    ]
)


Training until validation scores don't improve for 40 rounds
[10]	valid_0's ndcg@1: 1	valid_0's ndcg@3: 0.99999	valid_0's ndcg@5: 0.999988	valid_0's ndcg@10: 0.999984
[20]	valid_0's ndcg@1: 1	valid_0's ndcg@3: 1	valid_0's ndcg@5: 1	valid_0's ndcg@10: 1
[30]	valid_0's ndcg@1: 1	valid_0's ndcg@3: 1	valid_0's ndcg@5: 1	valid_0's ndcg@10: 1
[40]	valid_0's ndcg@1: 1	valid_0's ndcg@3: 0.99999	valid_0's ndcg@5: 0.999993	valid_0's ndcg@10: 0.999994
Early stopping, best iteration is:
[1]	valid_0's ndcg@1: 1	valid_0's ndcg@3: 1	valid_0's ndcg@5: 1	valid_0's ndcg@10: 1


In [37]:
X_val['pred_score'] = model.predict(X_val[feats]) 

C:\Users\Usuario\AppData\Local\Temp\ipykernel_14376\4280188630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['pred_score'] = model.predict(X_val[feats])


In [ ]:
print(y_val.head())

0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64


In [38]:
import numpy as np
from sklearn.metrics import ndcg_score

# Let’s assume: 
# - X_val has the same features you used for training.
# - y_val is the ground-truth label (0/1).
# - meta_val has the user_id information.

# Generate predicted scores for the validation set
pred_scores = model.predict(X_val[feats], num_iteration=model.best_iteration)

# Combine back into a DataFrame for easy groupby
val_df = X_val.copy()
val_df['pred_score'] = pred_scores
val_df['label'] = y_val.values
val_df['user_id'] = meta_val['user_id'].values

ndcg_list = []

for user_id, group in val_df.groupby('user_id'):
    if len(group) < 2:
        continue  # skip users with only 1 product

    true_relevance = group['label'].values.reshape(1, -1)
    scores = group['pred_score'].values.reshape(1, -1)
    
    if true_relevance.sum() > 0:
        ndcg = ndcg_score(true_relevance, scores, k=10)
        ndcg_list.append(ndcg)

mean_ndcg = np.mean(ndcg_list)
print(f"Mean NDCG@10: {mean_ndcg:.4f}")


Mean NDCG@10: 0.8186


In [39]:
from sklearn.metrics import ndcg_score
X_val['label'] = train_df['label'].values      
# Group back predictions by user
for user_id, group in X_val.groupby('user_id'):
    true_relevance = group['label'].values
    scores = group['pred_score'].values
    ndcg = ndcg_score([true_relevance], [scores], k=10)
    # store or print ndcg


In [40]:
print(ndcg)

0.907613572455749


In [ ]:
label_counts = training_df.groupby('user_id')['label'].nunique()
invalid_users = label_counts[label_counts < 2]
print(f"Users with invalid label variety: {len(invalid_users)}")


Users with invalid label variety: 0


In [41]:
y_val.value_counts(normalize=True) # nos aseguramos de que sea balanceado el data set


label
1    0.681762
0    0.318238
Name: proportion, dtype: float64